In [1]:
import pickle
import math
import numpy as np
import time
import networkx as nx

import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [4]:
AutoML_path = ""
log_folder = "tpot_log"

log_DDN = AutoML_path+log_folder+'/DDN_single_time'
log_MF = AutoML_path+log_folder+'/meta_features_02272022'
log_pipeline_steps = AutoML_path+log_folder+'/pipeline_steps_class_02272022_tpot'

combination_name = 'MF+GIN'
today = '03102022'
log_target_test_pred_time = AutoML_path+log_folder+'/'+combination_name+'_target_test_pred_time_'+today


In [6]:
DDN = pickle.load(open(log_DDN, "rb"))
print(len(DDN))

MF = pickle.load(open(log_MF, "rb"))
MF_keys = list(MF[list(MF.keys())[0]][0].keys())
MF_np = {}
for task in MF:
    feature = [MF[task][0][key]['value'] for key in MF_keys]
    feature = [0 if type(i) != float or math.isnan(i) else i for i in feature]
    MF_np[task] = np.array(feature)
print(len(MF_np))

pipeline_steps_class = pickle.load(open(log_pipeline_steps, 'rb'))
print(len(pipeline_steps_class))

pipeline_steps_class = [i for i in pipeline_steps_class if i[3] != "Timeout" and not np.isinf(i[3])]
print(len(pipeline_steps_class))


39
39
12863
11797


In [7]:
from torch_geometric.data import Data
from collections import defaultdict
device = torch.device('cpu')
graphs = defaultdict(list)

for pipeline in pipeline_steps_class:
    data_name = pipeline[0]
    if data_name in MF_np:
        g = pipeline[2]
        score = pipeline[3]
        node_attribute_dict = nx.get_node_attributes(g, name='feature')
        node_attribute_list = [np.hstack((node_attribute_dict[i],MF_np[data_name])) for i in node_attribute_dict]
#         node_attribute_list = [np.hstack((node_attribute_dict[i],X_meta[data_name])) for i in node_attribute_dict]
        node_x = torch.tensor(node_attribute_list, dtype=torch.float)

        edge_list = list(g.edges())
        edge_index = torch.tensor(edge_list,dtype=torch.long).t()

        graph_data = Data(edge_index=edge_index, x=node_x, y=score)
        graphs[data_name].append(graph_data.to(device))
        

<ipython-input-7-8d9bf76e7917>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  node_x = torch.tensor(node_attribute_list, dtype=torch.float)


In [8]:
print(len(graphs))
graphs['adult'][0]

39


Data(x=[46, 277], edge_index=[2, 1], y=0.7911666666666666)

In [9]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, GINConv
from torch_geometric.nn import global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GIN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GINConv(torch.nn.Sequential(
           Linear(in_channels, hidden_channels)
        ))
        self.conv2 = GINConv(torch.nn.Sequential(
           Linear(hidden_channels, hidden_channels)
        ))
        self.conv3 = GINConv(torch.nn.Sequential(
           Linear(hidden_channels, hidden_channels)
        ))
        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, 1)

    def forward(self, x, edge_index, batch):
        dataset_embedding = x[:, 40:104]
#         print(x.shape, dataset_embedding.shape)
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv3(x, edge_index)
        x = x.relu()

        # 2. Readout layer
#         x = torch.cat((x, dataset_embedding), dim=1)
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
#         x = F.dropout(x, p=0.2, training=self.training)
#         print(x.shape, dataset_embedding.shape)
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        x = x.relu()
        x = self.lin3(x)
        
        return x

In [16]:
from torch_geometric.data import DataLoader

target_test_pred_time = {}
data_names = list(MF_np.keys())
data_embed = MF_np
batch_size = 64

for data_index, data_name in enumerate(data_names):
    print("="*40)
    print(data_index, data_name)
    
    task_test = [data_name]
    candidate_score = DDN[data_name]
    prev_key = None
    dict_key = None
    len_task_train = 0
    prev_len_task_train = 0
    for threshold in np.arange(0.05, 0.5, 0.05):
        print("-"*40, threshold)
        prev_key = dict_key
        dict_key = '_'.join([data_name, combination_name, str(round(threshold,2))])
        
        start = time.time()
        task_train = [i for i in data_names if i!=data_name and abs(DDN[i]-candidate_score)<threshold]
        prev_len_task_train = len_task_train
        len_task_train = len(task_train)
        if len_task_train == 0:
            target_test_pred_time[dict_key] = 'N.A.'
            pickle.dump(target_test_pred_time, open(log_target_test_pred_time, "wb"))
            continue
            
        if len_task_train == prev_len_task_train:
            print("+"*30)
            print(prev_key, dict_key)
            print(prev_len_task_train, len_task_train)
            target_test_pred_time[dict_key] =  target_test_pred_time[prev_key]
            continue

        graphs_train = []
        graphs_test = []
        for key in graphs:
            if key in task_train:
                graphs_train += graphs[key]
            elif key in task_test:
                graphs_test += graphs[key]
        train_loader = DataLoader(graphs_train, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(graphs_test, batch_size=batch_size, shuffle=False)
        print("graphs_train shape:", len(graphs_train), "graphs_train[0] shape:", graphs_train[0])
        print("graphs_test shape:", len(graphs_test), "graphs_test[0] shape:", graphs_test[0])

        model = GIN(in_channels=graphs_train[0].num_node_features, hidden_channels=64).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
        criterion = torch.nn.MSELoss() 
        def train():
            model.train()
            for data in train_loader:  # Iterate in batches over the training dataset.
                out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
                loss = criterion(out, data.y.float().reshape(data.y.shape[0],1).to(device))  # Compute the loss.
#                 print(data.y.float())
                # loss = criterion(out, data.y.to(device))  # Compute the loss.
                loss.backward()  # Derive gradients.
                optimizer.step()  # Update parameters based on gradients.
                optimizer.zero_grad()  # Clear gradients.

        def test(loader):
            model.eval()
            test = []
            pred = []
            for data in loader:  # Iterate in batches over the training/test dataset.
                out = model(data.x, data.edge_index, data.batch)  
        #         print(out, data.y)
                if len(test) == 0:
                    test = data.y
                    pred = out.detach()
                else:
                    test = torch.cat((test, data.y))
                    pred = torch.cat((pred, out.detach()))
            return test, pred, mean_absolute_error(test.cpu(), pred.cpu()), mean_squared_error(test.cpu(), pred.cpu())  # Derive ratio of correct predictions.


        start = time.time()
        for epoch in range(10):
            train()
            _, _, train_mae, train_mse = test(train_loader)
            test_y, pred_y, test_mae, test_mse = test(test_loader)
        #     test_acc = test(test_loader)
        end  = time.time()
        duration = end-start
        print(f'Epoch: {epoch:03d}, Train MAE: {train_mae:.4f}, Train MSE: {train_mse:.4f}')
        print(f'Epoch: {epoch:03d}, Test MAE: {test_mae:.4f}, Test MSE: {test_mse:.4f}')
        print("Time:", duration)
        
        target_test_pred_time[dict_key] = (test_y, pred_y, duration, test_mae, test_mse)
        pickle.dump(target_test_pred_time, open(log_target_test_pred_time, 'wb'))
    


0 adult
---------------------------------------- 0.05
graphs_train shape: 2129 graphs_train[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.7552428795420986)
graphs_test shape: 271 graphs_test[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.7911666666666666)
Epoch: 009, Train MAE: 0.0902, Train MSE: 0.0140
Epoch: 009, Test MAE: 0.0554, Test MSE: 0.0046
Time: 6.777953147888184
---------------------------------------- 0.1
graphs_train shape: 4352 graphs_train[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.7552428795420986)
graphs_test shape: 271 graphs_test[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.7911666666666666)


/Users/haox/opt/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 009, Train MAE: 0.0757, Train MSE: 0.0103
Epoch: 009, Test MAE: 0.0410, Test MSE: 0.0036
Time: 12.857715129852295
---------------------------------------- 0.15000000000000002
graphs_train shape: 7632 graphs_train[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.932)
graphs_test shape: 271 graphs_test[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.7911666666666666)


/Users/haox/opt/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 009, Train MAE: 0.1073, Train MSE: 0.0227
Epoch: 009, Test MAE: 0.0733, Test MSE: 0.0085
Time: 22.16364097595215
---------------------------------------- 0.2
graphs_train shape: 8814 graphs_train[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.932)
graphs_test shape: 271 graphs_test[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.7911666666666666)


...

Epoch: 009, Train MAE: 0.1240, Train MSE: 0.0252
Epoch: 009, Test MAE: 0.2366, Test MSE: 0.0629
Time: 34.12738037109375
---------------------------------------- 0.35000000000000003
graphs_train shape: 9660 graphs_train[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.45183333333333336)
graphs_test shape: 357 graphs_test[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.30466666666666664)


/Users/haox/opt/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 009, Train MAE: 0.1315, Train MSE: 0.0257
Epoch: 009, Test MAE: 0.2631, Test MSE: 0.0762
Time: 43.091737270355225
---------------------------------------- 0.4
graphs_train shape: 11440 graphs_train[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.932)
graphs_test shape: 357 graphs_test[0] shape: Data(x=[46, 277], edge_index=[2, 1], y=0.30466666666666664)


/Users/haox/opt/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 009, Train MAE: 0.1716, Train MSE: 0.0565
Epoch: 009, Test MAE: 0.2914, Test MSE: 0.0919
Time: 52.55566883087158
---------------------------------------- 0.45
++++++++++++++++++++++++++++++
volkert_MF+GIN_0.4 volkert_MF+GIN_0.45
38 38


In [11]:
loss

NameError: name 'loss' is not defined